# Python sdk as Service Principal

## Overview

Details related to Databricks sdk for python are described: <a href="https://learn.microsoft.com/en-us/azure/databricks/dev-tools/sdk-python" target="_blank">here</a>

## Step 1: databricks sdk install

In [0]:
%pip install databricks-sdk --upgrade

In [0]:
dbutils.library.restartPython()

In [0]:
%pip show databricks-sdk | grep -oP '(?<=Version: )\S+'

## Step 2: Authentication
Details related to python authentication as Service Principal are described: <a href="https://learn.microsoft.com/en-us/azure/databricks/dev-tools/auth/oauth-m2m#python" target="_blank">here</a>

In [0]:
workspace_url = spark.conf.get("spark.databricks.workspaceUrl")
display(workspace_url)

### Manual Authentication Setup

In [0]:
dbutils.widgets.text(name='client_id', defaultValue='')
dbutils.widgets.text(name='token', defaultValue='')
dbutils.widgets.text(name='account_id', defaultValue='')

import os

sp_cid = dbutils.widgets.get('client_id')
sp_tkn = dbutils.widgets.get('token')
ws_url =  workspace_url
ac_url =  "https://accounts.azuredatabricks.net"
ac_id = dbutils.widgets.get('account_id')

In [0]:
display("sp_cid: " + sp_cid)
display("sp_tkn: " + sp_tkn)
display("ws_url: " + ws_url)
display("ac_url: " + ac_url)
display("ac_id: "  + ac_id)

### Auto Authentication Setup via Secrets

In [0]:
scopes = dbutils.secrets.listScopes()
scopes

In [0]:
secrets = dbutils.secrets.list("scp-ag83-cm-consumer-008")
secrets

In [0]:
dbutils.widgets.text(name='account_id', defaultValue='')

import os

sp_cid = dbutils.secrets.get(scope = "scp-ag83-cm-consumer-008", key = "scr-ag83-cm-consumer-admin-008-uuid")
sp_tkn = dbutils.secrets.get(scope = "scp-ag83-cm-consumer-008", key = "scr-ag83-cm-consumer-admin-008-token")
ws_url =  workspace_url
ac_url =  "https://accounts.azuredatabricks.net"
ac_id = dbutils.widgets.get('account_id')

In [0]:
display("sp_cid: " + sp_cid)
display("sp_tkn: " + sp_tkn)
display("ws_url: " + ws_url)
display("ac_url: " + ac_url)
display("ac_id: "  + ac_id)

### Account-level operations

In [0]:
from databricks.sdk import AccountClient

a = AccountClient(
  host          = ac_url,
  account_id    = ac_id,
  client_id     = sp_cid,
  client_secret = sp_tkn
)

### Workspace-level operations

In [0]:
from databricks.sdk import WorkspaceClient

w = WorkspaceClient(
  host          = ws_url,
  client_id     = sp_cid,
  client_secret = sp_tkn
)

## Step 3: Use the Sdk

### Account-level operations

In [0]:
for m in a.metastores.list():
    if m.name.startswith("mts-ag83") and not m.name.endswith("008"):
        print(m.name)

In [0]:
for m in a.metastores.list():
    if m.name.startswith("mts-ag83") and not m.name.endswith("008"):
        a.metastores.delete(m.metastore_id, force=True)

In [0]:
for ws in a.workspaces.list():
    if ws.workspace_name.startswith("dbw-ag83"):
        print(ws.workspace_name)

In [0]:
for sp in a.service_principals.list():
    if sp.display_name is not None and sp.display_name.startswith("sp-ag83") and not sp.display_name.endswith("008"):
        print(sp.display_name)

In [0]:
for g in a.groups.list():
  if g.display_name.startswith("grp-ag83") and not g.display_name.endswith("008"):
    print(g.display_name)

In [0]:
for p in a.budgets.list():
    if p.display_name.startswith("bdg-ag83") and not p.display_name.endswith("008"):
        print(p.display_name)

In [0]:
for p in a.budgets.list():
    if p.display_name.startswith("bdg-ag83") and not p.display_name.endswith("008"):
        a.budgets.delete(p.budget_configuration_id)

In [0]:
for p in a.budget_policy.list():
    print(p.policy_name)

### Workspace-level operations

In [0]:
for c in w.warehouses.list():
  print(c.name)

In [0]:
for p in w.budget_policy.list():
    print(p)